<a href="https://colab.research.google.com/github/Asmaa6777/Kepler-Exoplanet/blob/main/Kepler_Exoplanet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import *
from sklearn.metrics import log_loss, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.inspection import permutation_importance
from sklearn import set_config
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier


In [16]:
!git clone https://github.com/Asmaa6777/Kepler-Exoplanet.git

fatal: destination path 'Kepler-Exoplanet' already exists and is not an empty directory.


In [17]:
url = '/content/Kepler-Exoplanet/archive (9).zip'

df = pd.read_csv(url)



In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9564 entries, 0 to 9563
Data columns (total 50 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rowid              9564 non-null   int64  
 1   kepid              9564 non-null   int64  
 2   kepoi_name         9564 non-null   object 
 3   kepler_name        2294 non-null   object 
 4   koi_disposition    9564 non-null   object 
 5   koi_pdisposition   9564 non-null   object 
 6   koi_score          8054 non-null   float64
 7   koi_fpflag_nt      9564 non-null   int64  
 8   koi_fpflag_ss      9564 non-null   int64  
 9   koi_fpflag_co      9564 non-null   int64  
 10  koi_fpflag_ec      9564 non-null   int64  
 11  koi_period         9564 non-null   float64
 12  koi_period_err1    9110 non-null   float64
 13  koi_period_err2    9110 non-null   float64
 14  koi_time0bk        9564 non-null   float64
 15  koi_time0bk_err1   9110 non-null   float64
 16  koi_time0bk_err2   9110 

# **Data Preprocessing**

In [19]:
df = df.drop('kepler_name',axis=1)

In [20]:
#encode the data
X = df.copy()
X = X.drop('koi_disposition', axis=1)
y = df.koi_disposition.copy()
y = LabelEncoder().fit_transform(y)



dadicated = 0

comfirmed = 1

false positave = 2


In [21]:
#splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [28]:
# Define numerical and categorical columns
num_cols = X.select_dtypes(include='number').columns.tolist() # Select only numerical columns
cat_cols = X.select_dtypes(exclude='number').columns.tolist() # Select only categorical columns

# Define the imputer for numerical data
num_prep = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

# categorical preprocessor
cat_prep = Pipeline([
    ('imp', SimpleImputer(strategy='most_frequent')), # Use most_frequent for categorical
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # Add OneHotEncoder for categorical features
])

# Combine preprocessing steps
prepper = ColumnTransformer([
    ('numeric', num_prep, num_cols),
    ('categorical', cat_prep, cat_cols) # Include categorical preprocessing
], remainder='passthrough')

# Apply the preprocessing pipeline to the data
X_train_prepared = prepper.fit_transform(X_train)
X_test_prepared = prepper.transform(X_test)
X_val_prepared = prepper.transform(X_val)


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:558: UserWarning: Skipping features without any observed values: ['koi_teq_err1' 'koi_teq_err2']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:558: UserWarning: Skipping features without any observed values: ['koi_teq_err1' 'koi_teq_err2']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:558: UserWarning: Skipping features without any observed values: ['koi_teq_err1' 'koi_teq_err2']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


# **Modeling**

In [35]:
model = LGBMClassifier(random_state=42)
model.fit(X_train_prepared, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003834 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9525
[LightGBM] [Info] Number of data points in the train set: 6120, number of used features: 48
[LightGBM] [Info] Start training from score -1.456689
[LightGBM] [Info] Start training from score -1.417495
[LightGBM] [Info] Start training from score -0.644980
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMClassifier(random_state=42)

In [36]:
y_pred_val = model.predict(X_val_prepared)
accuracy_val = accuracy_score(y_val, y_pred_val)
print("Accuracy:", accuracy_val)

Accuracy: 0.9261920313520575


In [37]:
y_pred = model.predict(X_test_prepared)
accuracy = accuracy_score(y_test ,y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9194981704129639


**feature importance**

In [ ]:
# Get feature importance
importance = model.feature_importances_

# Get the feature names from the ColumnTransformer
feature_names = prepper.get_feature_names_out()

# Create a DataFrame to pair feature names with their importance scores
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importance})

# Sort the features by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display the feature importance
print(feature_importance_df)

                                Feature  Importance
0                         numeric__pca0         559
2                         numeric__pca2         553
3                         numeric__pca3         462
25                       numeric__pca25         445
17                       numeric__pca17         440
...                                 ...         ...
2067  categorical__kepoi_name_K02311.02           0
2066  categorical__kepoi_name_K02311.01           0
2065  categorical__kepoi_name_K02310.01           0
2064  categorical__kepoi_name_K02309.01           0
3075  categorical__kepoi_name_K03681.01           0

[6151 rows x 2 columns]


#  **Evaluation**

In [ ]:
# Accuracy
accuracy = accuracy_score(y_test, y_pred)

# Precision
precision = precision_score(y_test, y_pred, average='macro')

# Recall
recall = recall_score(y_test, y_pred, average='macro')

# F1 Score
f1 = f1_score(y_test, y_pred, average='macro')

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred2)

# Detailed classification report
print(classification_report(y_test, y_pred2))

# Print evaluation metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
#print(f"ROC-AUC: {roc_auc}")

              precision    recall  f1-score   support

           0       0.83      0.76      0.79       451
           1       0.77      0.82      0.79       454
           2       0.99      1.00      0.99      1008

    accuracy                           0.90      1913
   macro avg       0.86      0.86      0.86      1913
weighted avg       0.90      0.90      0.90      1913

Accuracy: 0.8991113434396236
Precision: 0.8620602446387112
Recall: 0.8577593925718993
F1 Score: 0.8591693335097332
